# Clio Muse Tours Plan Execution

## Project Structure & Execution
0. [**Discovery**](#discovery): Identify the business frame we will work on
1. [**Setup**](Assignment_Execution.ipynb#setup): Create folder structure and install dependencies
3. [**Explore Data**](#explore-data): Explore how data are structured in each file.
2. [**Cleaning**](#cleaning): Apply cleaning rules and validate processed data
3. [**Transformation**](#transformation): Create processed tables where we merge different dataset to achieve different outcomes
4. [**Analysis**](#analysis): Generate insights before clustering
5. [**Clustering**](#clustering): Segment data to provide further structured insights
6. [**Visualization**](#visualization): Create visualizations here or build PowerBI Dashboards
7. [**Reporting**](#reporting): Synthesize findings in a presentation


## Discovery

### Business Questions

* **Business Question**: Booking lead time analysis
    * **Analytical Objective**: Quantify time between booking and travel
    * **Success Metric**: Lead time distribution, median/mean
* **Business Question**: Refund rates by product
    * **Analytical Objective**: Identify high-risk products
    * **Success Metric**: Refund rate % by product_type, product_country
* **Business Question**: Group size vs refund behavior
    * **Analytical Objective**: Understand group booking patterns
    * **Success Metric**: Correlation coefficient, refund rate by group size
* **Business Question**: Content consumption depth
    * **Analytical Objective**: Measure engagement quality
    * **Success Metric**: Completion rates, average listening duration
* **Business Question**: Drop-off points
    * **Analytical Objective**: Identify content friction
    * **Success Metric**: Story-level abandonment rates
* **Business Question**: User segemntation
    * **Analytical Objective**: Create actionable personas
    * **Success Metric**: 3-5 distinct clusters with interpretable behaviors
---

> Visulizations need to enable decision making

### Constraints & Assumptions
- **Time Range Mismatch**: Bookings (2024) vs App data (2025) - analyze separately, note limitation
- **User Identity**: `user_id` (logged-in) vs `user_pseudo_id` (anonymous) - handle both
- **Missing Data**: Some events lack user_id - use pseudo_id for anonymous analysis
- **Timestamp Precision**: Microseconds - convert to datetime, handle timezone (assume UTC)
- **Language Mapping**: `language` field (e.g., "pt-pt") vs `lang_id` - reconcile via mapping

## Setup

Paste the following commands in the terminal and wait for them to be installed:
```
pip install pandas
pip install numpy
pip install scikit-learn
pip install matplotlib
pip install seaborn
pip install plotly
pip install jupyter
pip install ipykernel
pip install pyarrow
pip install openpyxl
```

In [11]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
import glob


## Explore Data

Let's start by loading the data and watching them at a high level.

For that we'll use glob to make our live easier and manually load the data for each month.

In [14]:
def load_many_csv(pattern, **read_csv_kwargs):
    paths = sorted(glob.glob(pattern))
    
    dfs = []

    for path in paths:
        df = pd.read_csv(path, **read_csv_kwargs)
        df['source_file'] = path.split("/")[-1]  # Add a column to identify the source file
        dfs.append(df)

    out = pd.concat(dfs, ignore_index=True)
    return out, paths

# load data: users, events, mappings, bookings
users_all, user_paths = load_many_csv("../Data/raw/users/users_data_2025-*.csv")
events_all, event_paths = load_many_csv("../Data/raw/events/events_data_2025-*.csv")
bookings_data = pd.read_csv("../Data/raw/bookings/travellers_data_2024.csv")
id_lang_mappings_df = pd.read_csv("../Data/raw/mappings/id_language_mapping.csv")
tour_title_mapping_df = pd.read_csv("../Data/raw/mappings/tour_title_mapping.csv")

print(f"Users files: {user_paths}")
print(f"Users shape: {users_all.shape}")
print("---" * 10)
print(f"Events files: {event_paths}")
print(f"Events shape: {events_all.shape}")
print("---" * 10)
print(f"Bookings shape: {bookings_data.shape}")
print("---" * 10)
print(f"ID Language Mappings shape: {id_lang_mappings_df.shape}")
print("---" * 10)
print(f"Tour Title Mappings shape: {tour_title_mapping_df.shape}")


C:\Users\konst\AppData\Local\Temp\ipykernel_13140\3220622363.py:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, **read_csv_kwargs)


Users files: ['../Data/raw/users\\users_data_2025-07.csv', '../Data/raw/users\\users_data_2025-08.csv', '../Data/raw/users\\users_data_2025-09.csv', '../Data/raw/users\\users_data_2025-10.csv']
Users shape: (52743, 6)
------------------------------
Events files: ['../Data/raw/events\\events_data_2025-07.csv', '../Data/raw/events\\events_data_2025-08.csv', '../Data/raw/events\\events_data_2025-09.csv', '../Data/raw/events\\events_data_2025-10.csv']
Events shape: (13471335, 13)
------------------------------
Bookings shape: (163828, 10)
------------------------------
ID Language Mappings shape: (22, 4)
------------------------------
Tour Title Mappings shape: (199, 2)


Okay, so we have:
- **Users:** 52743 records of users 
    - **Questions for us (the team):**
        - Are all the records corresponding to unique users?
- **Events:** 13,471,335 events recorded
    - **Questions for us (the team):**
        - Are all the records corresponding to an existing user?
- **Bookings:** 163828 bookigns for 2024
- **Id Language Mapping:** 22 Different Language supported
- **Tour Titles:** Currently supporting 199 tours inside the app.

---

Okay great, we have a certain high-level idea of our data now let's dig a little deeper.

We'll check all columns per dataset to confirm:
- `dtypes`
- `missingness`
- `uniqueness`
- `ranges`

For that we'll use a function that takes a dataframe, and returns for each dataset, a profiled dataframe of the columns.

In [41]:
def profile_df(df, name="df", top_n=5):
    prof = []
    for col in df.columns:
        s = df[col]
        dtype = str(s.dtype)
        missing = int(s.isna().sum())
        missing_pct = float(missing / len(df)) if len(df) else 0.0
        nunique = int(s.nunique(dropna=True))

        row = {
            "dataset": name,
            "column": col,
            "dtype": dtype,
            "rows": len(df),
            "missing": missing,
            "missing_pct": round(missing_pct, 4),
            "nunique": nunique
        }

        # Numeric stats
        if pd.api.types.is_numeric_dtype(s):
            row.update({
                "min": float(np.nanmin(s)),
                "p50": float(np.nanpercentile(s, 50)),
                "p99": float(np.nanpercentile(s, 99)),
                "max": float(np.nanmax(s))
            })

        # Datetime stats
        if pd.api.types.is_datetime64_any_dtype(s):
            row.update({
                "min_dt": str(s.min()),
                "max_dt": str(s.max())
            })

        prof.append(row)

    prof_df = pd.DataFrame(prof).sort_values(["missing_pct","nunique"], ascending=[False, False])
    return prof_df

# Profile the loaded DataFrames
print(profile_df(users_all, name="Users Data"))
print(profile_df(events_all, name="Events Data"))
# print(profile_df(bookings_data, name="Bookings Data"))
# print(profile_df(id_lang_mappings_df, name="ID Language Mappings"))
# print(profile_df(tour_title_mapping_df, name="Tour Title Mappings"))

      dataset                             column                dtype   rows  \
1  Users Data                first_purchase_date       datetime64[ns]  52743   
0  Users Data                            user_id                int64  52743   
2  Users Data  user_first_touch_timestamp_micros                int64  52743   
6  Users Data                 first_touch_ts_utc  datetime64[ns, UTC]  52743   
4  Users Data                            country               string  52743   
5  Users Data                        source_file               object  52743   
3  Users Data                   operating_system               string  52743   

   missing  missing_pct  nunique           min           p50           p99  \
1    52216         0.99       96           NaN           NaN           NaN   
0        0         0.00    27171  0.000000e+00  5.325760e+05  5.812077e+05   
2        0         0.00     1043  1.666224e+15  1.752797e+15  1.761523e+15   
6        0         0.00     1043           NaN 

TypeError: numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.

How we can intepret results here:
**Users Data – Key Findings**
- `first_purchase_date` is almost entirely missing (≈99%)
    - Any purchase-related analysis using this field must, clearly state that purchase data coverage is extremely limited

- `user_id` is complete and highly unique
    - 52,743 rows with 0% missing
    - ~27k unique user IDs → users appear multiple times across months
    - Confirms the need for deduplication into a dim_user table

- `user_first_touch_timestamp_micros` is complete
    - 0% missing which means we have a strong foundation for time-to-first-action metrics
    - Microsecond values confirm high-precision timestamps that must be converted to UTC datetime

- Country and OS are fully populated
    - `country`: 151 unique values, wide geographic spread
    - `operating_system`: only 2 values, iOS vs Android
    - Enables strong country-level and OS-level segmentation

**Events Data – Key Findings**

- Audio interaction fields are mostly missing
    - `audio_time_played`: ~99.6% missing
    - `audio_time_paused`: ~99.3% missing
    - You cannot assume every event represents listening behavior

- User identity is split between logged-in and anonymous
    - `user_id` missing in ~41% of events
    - `user_pseudo_id` has 0% missing
    - So what we need to is:
        - Treat anonymous and logged-in users separately
        - Full enrichment (country, OS) is only possible for logged-in users

- Event timestamps are complete and precise
    - `event_timestamp` has 0% missing and ~13.47M unique values

- Event date is numeric, not datetime
    - `event_date` is stored as an integer (YYYYMMDD)
    - Must be converted and validated against event_timestamp


- Tour and story coverage is broad but incomplete
    - `tour_id`: ~5% missing → some events are not tied to tours
    - `story_id`: ~20% missing → not all events are story-level
    - Analysis must:
        - filter to tour/story-related events when measuring listening depth
        - avoid assuming all events represent content consumption

- Language complexity exists
    - `language` has 416 unique values (e.g. pt-pt, en-us)
    - `lang_id` only has 9 unique values
    - Confirms need for:
        - normalization of app language
        - reconciliation with lang_id via mapping

---

Okay now that we know our data very well, and understand  let's return to confirm our assumptions now that we know a lot more about our data

1. Time Range Mismatch (2024 bookings vs 2025 events/users)
    - For that we'll first standardize timestamps from milliseconds, to actual datetimes
    - For `users_all`, we'll standardize `first_purchase_date` by transforming it into datetime if available and `user_first_touch_timestamp_micros` where we are adding a new column with standardize datetime
    - The same process goes on for events and bookings too

In [32]:
def standardize_users(users_df):
    df = users_df.copy()

    # Dates
    df["first_purchase_date"] = pd.to_datetime(df["first_purchase_date"], errors="coerce")

    # Micros → datetime UTC
    df["first_touch_ts_utc"] = pd.to_datetime(
        df["user_first_touch_timestamp_micros"],
        unit="us",
        utc=True,
        errors="coerce"
    )

    # Clean categories
    df["operating_system"] = df["operating_system"].astype("string").str.strip()
    df["country"] = df["country"].astype("string").str.strip()

    return df

users_all = standardize_users(users_all)
print("Users standardized dtypes:")
print(users_all.dtypes)

def standardize_bookings(bookings_df):
    df = bookings_df.copy()

    df["booking_date"] = pd.to_datetime(df["booking_date"], errors="coerce")
    df["travel_date"] = pd.to_datetime(df["travel_date"], errors="coerce")

    # Group size
    df["num_of_travellers"] = pd.to_numeric(df["num_of_travellers"], errors="coerce")

    # Basic cleanup
    for c in ["product_code","product_title","product_type","product_country","booking_status","travel_slot"]:
        if c in df.columns:
            df[c] = df[c].astype("string").str.strip()

    return df

bookings = standardize_bookings(bookings_data)
print("Bookings standardized dtypes:")
print(bookings.dtypes)  

def standardize_events(events_df):
    df = events_df.copy()

    df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce")

    # Micros → datetime UTC
    df["event_ts_utc"] = pd.to_datetime(
        df["event_timestamp"],
        unit="us",
        utc=True,
        errors="coerce"
    )

    # Clean categories
    df["event_name"] = df["event_name"].astype("string").str.strip()
    df["platform"] = df["platform"].astype("string").str.strip()

    return df
events_all = standardize_events(events_all)
print("Events standardized dtypes:")
print(events_all.dtypes)

Users standardized dtypes:
user_id                                            int64
first_purchase_date                       datetime64[ns]
user_first_touch_timestamp_micros                  int64
operating_system                          string[python]
country                                   string[python]
source_file                                       object
first_touch_ts_utc                   datetime64[ns, UTC]
dtype: object
Bookings standardized dtypes:
booking_id                    int64
travel_date          datetime64[ns]
travel_slot          string[python]
booking_date         datetime64[ns]
product_code         string[python]
product_title        string[python]
product_type         string[python]
product_country      string[python]
num_of_travellers             int64
booking_status       string[python]
dtype: object
Events standardized dtypes:
event_date                datetime64[ns]
event_timestamp                    int64
event_name                string[python]
platf

The users dataset is now cleanly typed and analysis-ready, with correct datetime parsing and categorical normalization.

The transformation confirms that temporal user behavior analysis is feasible, while monetization analysis is limited due to sparse purchase data.

---

Okay now the timestamps are ready standardized into datetime for all necessary dfs:
- events_all
- users_all
- bookings_data

Now let's just use min/max to prove what we were thinking about our time range mismatch, and see if we can in any way connect bookings with users/events data

In [35]:
print("Bookings date range:",
      bookings_data["booking_date"].min(), "→", bookings_data["booking_date"].max(),
      "| Travel:", bookings_data["travel_date"].min(), "→", bookings_data["travel_date"].max())

print("Users first_touch range:",
      users_all["first_touch_ts_utc"].min(), "→", users_all["first_touch_ts_utc"].max())

print("Events event_date range:",
      events_all["event_ts_utc"].min(), "→", events_all["event_ts_utc"].max())

Bookings date range: 2023-01-31 → 2025-04-10 | Travel: 2024-01-01 → 2024-12-31
Users first_touch range: 2022-10-20 00:00:00+00:00 → 2025-10-31 00:00:00+00:00
Events event_date range: 2025-06-30 21:00:33.442000+00:00 → 2025-10-31 21:58:37.367000+00:00


Okay how do we interpet these data:
- The bookings dataset represents a complete travel year (2024), with bookings occurring both before and after the travel period, enabling robust lead-time and seasonality analysis.

- The app events dataset captures user engagement during July–October 2025, and is analyzed independently from bookings due to non-overlapping timeframes.

- User onboarding data spans multiple years and provides historical context for engagement patterns observed in 2025.

---

2. **User Identity**: `user_id` (logged-in) vs `user_pseudo_id` (anonymous) - handle both
- for that we have a function to identify how many rows have only `user_pseudo_id` and how many have both

In [36]:
def identity_breakdown(events_df):
    df = events_df.copy()

    has_user = df["user_id"].notna()
    has_pseudo = df["user_pseudo_id"].notna()

    df["identity_type"] = np.select(
        [has_user & has_pseudo, has_user & ~has_pseudo, ~has_user & has_pseudo],
        ["both", "logged_in_only", "anonymous_only"],
        default="no_id"
    )

    return df["identity_type"].value_counts(dropna=False)

# Example usage after you load events_all:
identity_breakdown(events_all)


identity_type
both              7945646
anonymous_only    5525689
Name: count, dtype: int64

Okay how do we interpet these data:

- The events dataset contains two dominant identity states:
    - events where both user_id and user_pseudo_id are present
    - events where only user_pseudo_id is present
- There are no events with only user_id and no events with missing identity entirely.

- `user_pseudo_id` is the primary tracking backbone
    - Present in 100% of events
    - Enables:
        - sessionization for all users
        - anonymous engagement analysis
        - pre-login behavior tracking
- `user_id` acts as an enrichment layer
    - Appears only after authentication
    - Adds:
        - geographic context
        - OS metadata
        - user lifecycle timestamps
    - Its absence does not block behavioral analysis, only enrichment
- Logged-in users still retain their pseudo identity
    - Confirms that:
        - authentication does not reset device tracking
        - behavior before and after login can be stitched via pseudo_id
    - This opens the door to:
        - login impact analysis
        - transition analysis (anonymous → logged-in)
---

3. **Missing Data**: Some events lack user_id - use pseudo_id for anonymous analysis
    - For that we'll have a function which takes our major key columns, and output the percentage of events that don't have the corresponding column

In [37]:
def missing_key_fields(events_df):
    keys = ["event_date","event_timestamp","event_name","tour_id","story_id","user_id","user_pseudo_id","lang_id"]
    existing = [k for k in keys if k in events_df.columns]
    return (events_df[existing].isna().mean().sort_values(ascending=False) * 100).round(2)

missing_key_fields(events_all)


user_id            41.02
story_id           20.41
lang_id             5.95
tour_id             4.70
event_name          0.00
event_timestamp     0.00
event_date          0.00
user_pseudo_id      0.00
dtype: float64

Okay how do we interpet these data:

- Missing data in the events dataset is structured and expected, not random.
- Core tracking fields are fully populated, while content- and identity-related fields are conditionally missing depending on event type and user state.
- This validates the hypothesis that user_pseudo_id must be used as the backbone identifier, with user_id used only when available.

- `user_id` - 41.02% events don't have `user_id`
    - Any analysis requiring demographics must be restricted to the 59% logged in subset
    - We don't need to drop these rows but instead we can rely on `user_pseudo_id`

- `story_id` - 20.41% events doesn't have a story_id
    - Many events are:
        - navigation events
        - app lifecycle events
        - non-audio interactions
    - **The problem**: we can't treat all events as story-level interactions.

- `lang_id` - 5.95% events don't have language info
    - analysis of language should stick to non-null `lang_id`

- `tour_id` - 4.70% events don't have a tour 
    - Tour level metrics must
        - filter to non-null `tour_id`
        - avoid assuming all events relate to content consumption

### How do we proceed for cleaning?

1. For events:
    - We only need `event_ts_utc` as our only authoritative time field
    - Not all events are listening behavior
2. For users:
    - `user_pseudo_id` is our primary behavioral key
    - `user_id` is used only for enrichment if present
    - dedup users across monthly files by grabbing the user's first touch timestamp, so that we will have one row per user
3. Remove null ids of `tour_id` `story_id` `lang_id` and turn the present ids to integers.
4. Clean Bookings Data

## Cleaning

After data clean we need to end up with the following tables
- `dim_users`: one row per user, enriched, deduplicated
- `events_clean`: all events, cleaned timestamps, identity logic applied
- `listening_events`: subset of events eligible for audio analysis
- `bookings_clean`: booking with lead time, cleaned categories.

## Transformation

## Analysis

## Clustering

## Visualization

## Reporting